# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
%matplotlib inline 

## imports for Python, Pandas

In [3]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [5]:
# load json as string
l = json.load((open('data/world_bank_projects_less.json')))
print type(l)

<type 'list'>


In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{u'$oid': u'52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{u'Percent': 100, u'Name': u'Education for all'}","[{u'code': u'65', u'name': u'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{u'$oid': u'52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{u'Percent': 30, u'Name': u'Other economic man...","[{u'code': u'24', u'name': u'Other economic ma...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [17]:
json_df = pd.read_json('data/world_bank_projects.json')
json_df.shape
json_df1 = json_df.copy()

In [18]:
json_df.keys()

Index([u'_id', u'approvalfy', u'board_approval_month', u'boardapprovaldate',
       u'borrower', u'closingdate', u'country_namecode', u'countrycode',
       u'countryname', u'countryshortname', u'docty',
       u'envassesmentcategorycode', u'grantamt', u'ibrdcommamt', u'id',
       u'idacommamt', u'impagency', u'lendinginstr', u'lendinginstrtype',
       u'lendprojectcost', u'majorsector_percent', u'mjsector_namecode',
       u'mjtheme', u'mjtheme_namecode', u'mjthemecode', u'prodline',
       u'prodlinetext', u'productlinetype', u'project_abstract',
       u'project_name', u'projectdocs', u'projectfinancialtype',
       u'projectstatusdisplay', u'regionname', u'sector', u'sector1',
       u'sector2', u'sector3', u'sector4', u'sector_namecode', u'sectorcode',
       u'source', u'status', u'supplementprojectflg', u'theme1',
       u'theme_namecode', u'themecode', u'totalamt', u'totalcommamt', u'url'],
      dtype='object')

### JSON exercise number ONE
1. Find the 10 countries with most projects

In [19]:
# there are 500 records
print json_df.shape[0]

# if there are 500 unique project_names then there must be one UNIQUE project per row
assert json_df.shape[0] == len(set(json_df['project_name']))

# so get a count for each distinct countryname. 
top10_vc = json_df.countryname.value_counts().head(10).reset_index()

# alternatively count the project_names per country.
top10_gb = json_df.groupby(by='countryname').count()['project_name'] \
       .sort_values(ascending=False).head(10).reset_index()

top10_vc.columns = ['countryname','project_count']
top10_gb.columns = ['countryname','project_count']

# sort so records in both Series' are in the same order for comparison.
top10_vc = top10_vc.sort_values(by='countryname').set_index('countryname')
top10_gb = top10_gb.sort_values(by='countryname').set_index('countryname')

assert (top10_vc == top10_gb).all().all()
print "Assert passed. The correct answer is:"
top10_vc.sort_values(by='project_count', ascending=False)


500
Assert passed. The correct answer is:


,project_count
countryname,
People's Republic of China,19
Republic of Indonesia,19
Socialist Republic of Vietnam,17
Republic of India,16
Republic of Yemen,13
Kingdom of Morocco,12
Nepal,12
People's Republic of Bangladesh,12
Africa,11


### JSON exercise number TWO
2. Find the top 10 major project themes (using column 'mjtheme_namecode')

In [20]:
def get_theme_counts(mjtheme_namecode):

    theme1_count = defaultdict(int)
    for k in mjtheme_namecode:
        for a in k:
            if a['name'] != '':
                f = ('0'+a['code'])[-2:]+'_'+a['name']
                theme1_count[f] += 1
            else:
                dummy = ('0'+str(int(a['code'])))[-2:]+'_blank_space'
                theme1_count[ dummy ] += 1
    blist = [(v,k) for k,v in theme1_count.items()]
    return sorted(blist,reverse=True)
top10list = get_theme_counts(json_df.mjtheme_namecode)
top10list[:10]

[(223, u'11_Environment and natural resources management'),
 (202, u'10_Rural development'),
 (197, u'08_Human development'),
 (184, u'02_Public sector governance'),
 (158, u'06_Social protection and risk management'),
 (130, u'04_Financial and private sector development'),
 (119, u'07_Social dev/gender/inclusion'),
 (72, u'05_Trade and integration'),
 (47, u'09_Urban development'),
 (33, u'01_Economic management')]

### JSON exercise number THREE
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [21]:
nlist = get_theme_counts(json_df.mjtheme_namecode)

# dict to act as lookup to get correct name for every theme code.
nm_dict = {str(int(nm[1].split('_')[0])): nm[1].split('_')[1] 
              for nm in sorted(nlist,reverse=True) if nm[1].split('_')[1] <> 'blank' }
nm_dict

{'1': u'Economic management',
 '10': u'Rural development',
 '11': u'Environment and natural resources management',
 '2': u'Public sector governance',
 '3': u'Rule of law',
 '4': u'Financial and private sector development',
 '5': u'Trade and integration',
 '6': u'Social protection and risk management',
 '7': u'Social dev/gender/inclusion',
 '8': u'Human development',
 '9': u'Urban development'}

In [22]:
flist = []
new_df = json_df[['countryname','mjtheme_namecode']]

# adds correct name for each code from lookup nm_dict..
for k in json_df.mjtheme_namecode:
    tmp = []
    for l in k:
        tmp.append({ u'code': l['code'], u'name': nm_dict[l['code']]})
    flist.append(tmp)

print len(flist)
new_df['mjtheme_namecode2'] = pd.Series(flist, index=json_df.index)      
        

500


/Users/chadleonard/miniconda2/envs/tutorial/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
oldcode = []
newcode = []
test = []

# quick and dirty test designed to ensure that mjtheme_namecode and mjtheme_namecode2 line up ...
# by line up, I mean that mjtheme_namecode2 was added to the df for the correct records ...
# compares the codes for every record in both columns ...
for l in new_df[['mjtheme_namecode']].values:
    for a in l:
        for b in a:
            oldcode.append(b['code'])
for l in new_df[['mjtheme_namecode2']].values:
    for a in l:
        for b in a:
            newcode.append(b['code'])
print len(newcode)
print len(oldcode)
assert newcode == oldcode
print "Assert passed: lists must match"


1499
1499
Assert passed: lists must match


In [24]:
oldlist = get_theme_counts(new_df.mjtheme_namecode)
newlist = get_theme_counts(new_df.mjtheme_namecode2)
newc = 0
oldc = 0

# compare old counts where name had some blanks to new counts where name has no blanks.
for c in newlist:
    nm = c[1].split('_')
    print 'new ==> code: '+ str(int(nm[0])) + '; name: ' + nm[1] + '; count: '+ str(c[0])
    newc = c[0]
    for a in oldlist:
        nm2 = a[1].split('_')
        if nm == nm2:
            print 'old ==> code: '+ str(int(nm2[0])) + '; name: ' + nm2[1] + '; count: '+ str(a[0])
            oldc = a[0]
        elif nm2[1] == 'blank' and nm2[0] == nm[0]:
            print 'old ==> code: '+ str(int(nm2[0])) + '; name: ' + nm2[1] + '; count: '+ str(a[0])
            # test: assert that the count of old non-blank names and blank names equals new count
            assert newc == oldc + a[0]
            print str(newc) + ' == ' + str(oldc) + ' + ' + str(a[0])
            print '=' * 10

new ==> code: 11; name: Environment and natural resources management; count: 250
old ==> code: 11; name: Environment and natural resources management; count: 223
old ==> code: 11; name: blank; count: 27
250 == 223 + 27
new ==> code: 10; name: Rural development; count: 216
old ==> code: 10; name: Rural development; count: 202
old ==> code: 10; name: blank; count: 14
216 == 202 + 14
new ==> code: 8; name: Human development; count: 210
old ==> code: 8; name: Human development; count: 197
old ==> code: 8; name: blank; count: 13
210 == 197 + 13
new ==> code: 2; name: Public sector governance; count: 199
old ==> code: 2; name: Public sector governance; count: 184
old ==> code: 2; name: blank; count: 15
199 == 184 + 15
new ==> code: 6; name: Social protection and risk management; count: 168
old ==> code: 6; name: Social protection and risk management; count: 158
old ==> code: 6; name: blank; count: 10
168 == 158 + 10
new ==> code: 4; name: Financial and private sector development; count: 146


### JSON exercises TWO and THREE via an alternative way

In [4]:
from collections import Counter
df = pd.read_json('data/world_bank_projects.json')
df.shape

(500, 50)

In [5]:
#another way to answer question 2
def tcount(tlist):
    return [t['code'] for t in tlist]

# use apply function to pass lists of mjtheme_namecode dicts to tcount to get all 
# of the codes ...
biglist = [item for sublist in df.mjtheme_namecode.apply(tcount) for item in sublist]

# use Counter on biglist to return the code and the number of times it appears in the list 
new_list = [(v,k) for k,v in dict(Counter(biglist)).items()]
sorted(new_list,reverse=True)[:10]


[(250, u'11'),
 (216, u'10'),
 (210, u'8'),
 (199, u'2'),
 (168, u'6'),
 (146, u'4'),
 (130, u'7'),
 (77, u'5'),
 (50, u'9'),
 (38, u'1')]

In [6]:
#another way to create name lookup dict ...
#create name dict lookup to use to fill blank names 
def get_nm_dict(tlist):
    return [(t['code'], t['name']) for t in tlist if t['name'] != '']

# turned tuples into strings to use the 'set' function to get rid of dupes ...
ldict = [str(i) for s in df.mjtheme_namecode.apply(get_nm_dict) for i in s]
lst = []

# 'eval' function used to turn the strings back into tuples. Need tuples to create dict.
for l in list(set(ldict)):
    l = eval(l)
    lst.append(l)
ndict = dict(lst)
ndict

{u'1': u'Economic management',
 u'10': u'Rural development',
 u'11': u'Environment and natural resources management',
 u'2': u'Public sector governance',
 u'3': u'Rule of law',
 u'4': u'Financial and private sector development',
 u'5': u'Trade and integration',
 u'6': u'Social protection and risk management',
 u'7': u'Social dev/gender/inclusion',
 u'8': u'Human development',
 u'9': u'Urban development'}

In [7]:
# for each code lookup and return the name from ndict. 
def fill_name(tlist):
    return [{ u'code': t['code'], u'name': ndict[t['code']]} for t in tlist ]


fdict = [s for s in df.mjtheme_namecode.apply(fill_name)]
df['mjtheme_namecode_complete'] = pd.Series(fdict, index=df.index)

df[['mjtheme_namecode','mjtheme_namecode_complete']].head(1)

,mjtheme_namecode,mjtheme_namecode_complete
0,"[{u'code': u'8', u'name': u'Human development'...","[{u'code': u'8', u'name': u'Human development'..."


In [34]:
# test that the correct value was returned to 'name' key ...
n=0
for theme in df['mjtheme_namecode_complete'].values:
    for dct in theme:
        assert dct['name'] == ndict[dct['code']]
        n+=1
print n
print

n=0
x=0
for theme in df['mjtheme_namecode'].values:
    for dct in theme:
        try:
            assert dct['name'] == ndict[dct['code']]
            n+=1
        except:
            assert dct['name'] == ''
            x+=1

print n
print x
print n+x

sum([a[0] for a in top10list if 'blank' in a[1]])

1499

1377
122
1499


122

In [217]:
oldcode = []
newcode = []

for l in df[['mjtheme_namecode']].values:
    for a in l:
        for b in a:
            #print b['name']
            oldcode.append(b['code'])
#print '+' * 20
for l in df[['mjtheme_namecode_complete']].values:
    for a in l:
        for b in a:
            #print b['name']
            newcode.append(b['code'])

print len(newcode)
print len(oldcode)
assert newcode == oldcode
print "Assert passed: lists must match"


1499
1499
Assert passed: lists must match
